In [1]:
import random
import numpy
import theano
import model
import sys
import functools
import time
from scipy.ndimage import filters

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5105)
/home/ubuntu/miniconda2/envs/theano/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def iterate_minibatches(dataset, batch_size=512):
    random.shuffle(dataset)
    for offset in xrange(0, len(dataset), batch_size):
        s = min(batch_size, len(dataset) - offset)
        batch_fonts = numpy.zeros((s,), dtype=numpy.int32)
        batch_chars = numpy.zeros((s,), dtype=numpy.int32)
        batch_ds = numpy.zeros((s, wh), dtype=theano.config.floatX)
        for z in xrange(s):
            i, j = dataset[offset + z]
            batch_fonts[z] = i
            batch_chars[z] = j
            m = filters.gaussian_filter(data[i][j], sigma=random.random()*1.0) # data augmentation
            batch_ds[z] = m.flatten() * 1. / 255

        yield s, 1.0 * offset / len(dataset), batch_fonts, batch_chars, batch_ds

        
def iterate_run(dataset, fn, tag):
    total_loss, total_reg, total_count = 0, 0, 0
    for s, progress, input_font, input_char, output in iterate_minibatches(dataset):
        t0 = time.time()
        loss, reg = fn(input_font, input_char, output)
        t = time.time() - t0
        total_loss += float(loss) * s
        total_reg += float(reg) * s
        total_count += s
        sys.stdout.write('%s: %6.2f%%, perf: %.6f + %.6f (last minibatch: %.6f + %.6f, %.3fs)\r' % (tag, 100.0 * progress, total_loss / total_count, total_reg / total_count, float(loss), float(reg), t))
        sys.stdout.flush()

    sys.stdout.write('\n')
    return total_loss / total_count

In [3]:
data = model.get_data()

In [4]:
n, k = data.shape[0], data.shape[1]
wh = data.shape[2] * data.shape[3]

In [5]:
model = model.Model(n, k, wh)
model.try_load()

float32
loading model...


In [6]:
train_fn_w_learning_rate = model.get_train_fn()

compiling training fn


In [7]:
test_fn = model.get_test_fn()

compiling testing fn


In [8]:
run_fn = model.get_run_fn()

In [9]:
train_set, test_set = model.sets()

In [10]:
print 'training...'
for learning_rate in [1.0, 0.3, 0.1, 0.03, 0.01]:
    epoch, last_loss = 0, float('inf')
    while True:
        print 'epoch', epoch, 'learning rate', learning_rate
        train_fn = functools.partial(train_fn_w_learning_rate, learning_rate)
        iterate_run(train_set, train_fn, 'train')
        loss = iterate_run(test_set, test_fn, 'test ')
        print("loss %f" %loss)
        if loss > last_loss and epoch > 3:
            break # decrease learning rate
        last_loss = loss
        model.save()
        epoch += 1

training...
epoch 0 learning rate 0.1


loss 0.074229
saving model...
epoch 1 learning rate 0.1


loss 0.074231
saving model...
epoch 2 learning rate 0.1


loss 0.074104
saving model...
epoch 3 learning rate 0.1


loss 0.074128
saving model...
epoch 4 learning rate 0.1


loss 0.074118
saving model...
epoch 5 learning rate 0.1


loss 0.074054
saving model...
epoch 6 learning rate 0.1


loss 0.074011
saving model...
epoch 7 learning rate 0.1


loss 0.074002
saving model...
epoch 8 learning rate 0.1


loss 0.073985
saving model...
epoch 9 learning rate 0.1


loss 0.074002
epoch 0 learning rate 0.03


loss 0.073958
saving model...
epoch 1 learning rate 0.03


loss 0.073945
saving model...
epoch 2 learning rate 0.03


loss 0.073978
saving model...
epoch 3 learning rate 0.03


loss 0.073959
saving model...
epoch 4 learning rate 0.03


loss 0.073926
saving model...
epoch 5 learning rate 0.03


loss 0.073921
saving model...
epoch 6 learning rate 0.03


loss 0.073907
saving model...
epoch 7 lea

KeyboardInterrupt: 

In [ ]:
print 'training...'
for learning_rate in [1.0, 0.3, 0.1, 0.03, 0.01]:
    epoch, last_loss = 0, float('inf')
    while True:
        print 'epoch', epoch, 'learning rate', learning_rate
        train_fn = functools.partial(train_fn_w_learning_rate, learning_rate)
        iterate_run(train_set, train_fn, 'train')
        loss = iterate_run(test_set, test_fn, 'test ')
        print("loss %f" %loss)
        if loss > last_loss and epoch > 3:
            break # decrease learning rate
        last_loss = loss
        model.save()
        epoch += 1

training...
epoch 0 learning rate 1.0


loss 0.074849
saving model...
epoch 1 learning rate 1.0


loss 0.073718
saving model...
epoch 2 learning rate 1.0


loss 0.072789
saving model...
epoch 3 learning rate 1.0


loss 0.071707
saving model...
epoch 4 learning rate 1.0


loss 0.071026
saving model...
epoch 5 learning rate 1.0


loss 0.070240
saving model...
epoch 6 learning rate 1.0


loss 0.069701
saving model...
epoch 7 learning rate 1.0


loss 0.069079
saving model...
epoch 8 learning rate 1.0


loss 0.068495
saving model...
epoch 9 learning rate 1.0


loss 0.068081
saving model...
epoch 10 learning rate 1.0

loss 0.067449
saving model...
epoch 11 learning rate 1.0


loss 0.066961
saving model...
epoch 12 learning rate 1.0


loss 0.066516
saving model...
epoch 13 learning rate 1.0


loss 0.066041
saving model...
epoch 14 learning rate 1.0


loss 0.065604
saving model...
epoch 15 learning rate 1.0


loss 0.065316
saving model...
epoch 16 learning rate 1.0


loss 0.064997
saving model

KeyboardInterrupt: 

In [ ]:
print 'training...'
for learning_rate in [1.0, 0.3, 0.1, 0.03, 0.01]:
    epoch, last_loss = 0, float('inf')
    while True:
        print 'epoch', epoch, 'learning rate', learning_rate
        train_fn = functools.partial(train_fn_w_learning_rate, learning_rate)
        iterate_run(train_set, train_fn, 'train')
        loss = iterate_run(test_set, test_fn, 'test ')
        print("loss %f" %loss)
        if loss > last_loss and epoch > 3:
            break # decrease learning rate
        last_loss = loss
        model.save()
        epoch += 1

training...
epoch 0 learning rate 1.0


loss 0.064243
saving model...
epoch 1 learning rate 1.0


loss 0.063967
saving model...
epoch 2 learning rate 1.0


loss 0.063709
saving model...
epoch 3 learning rate 1.0


loss 0.063458
saving model...
epoch 4 learning rate 1.0


loss 0.063176
saving model...
epoch 5 learning rate 1.0


loss 0.062975
saving model...
epoch 6 learning rate 1.0


loss 0.062768
saving model...
epoch 7 learning rate 1.0


loss 0.062544
saving model...
epoch 8 learning rate 1.0


loss 0.062310
saving model...
epoch 9 learning rate 1.0


loss 0.062109
saving model...
epoch 10 learning rate 1.0


loss 0.061922
saving model...
epoch 11 learning rate 1.0


loss 0.061718
saving model...
epoch 12 learning rate 1.0


loss 0.061554
saving model...
epoch 13 learning rate 1.0


loss 0.061429
saving model...
epoch 14 learning rate 1.0


loss 0.061263
saving model...
epoch 15 learning rate 1.0


loss 0.061043
saving model...
epoch 16 learning rate 1.0


loss 0.060912
saving mode